### Import Libraries 

In [1]:
import arcpy 
import os
import pandas as pd
import UIGeocoder as UIG
import UIProximity as UIP
import UIImpervious as UIM
import UIAggregate as UIA

In [2]:
path =  "C:\\Users\\padu\\Desktop\\UrbanInstitute\\Proximity Variables\\"
os.chdir(path)

In [3]:
GISproject = r"C:\\Users\\padu\\Desktop\\UrbanInstitute\\Proximity Variables\\Proximity Variables.aprx"
ProjectGDB =  r"C:\\Users\\padu\\Desktop\\UrbanInstitute\\Proximity Variables.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = ProjectGDB

### Geocoding

### Preview CSV to be Geocode and check the column names you would use

In [4]:
Banks = pd.read_csv('Bank.csv')
Banks.columns

Index(['ADDRESS', 'BKCLASS', 'CBSA', 'CBSA_DIV', 'CBSA_DIV_FLG', 'CBSA_DIV_NO',
       'CBSA_METRO', 'CBSA_METRO_FLG', 'CBSA_METRO_NAME', 'CBSA_MICRO_FLG',
       'CBSA_NO', 'CERT', 'CITY', 'COUNTY', 'CSA', 'CSA_FLG', 'CSA_NO',
       'ESTYMD', 'FI_UNINUM', 'MAINOFF', 'NAME', 'OFFNAME', 'OFFNUM',
       'RUNDATE', 'SERVTYPE', 'STALP', 'STCNTY', 'STNAME', 'UNINUM', 'ZIP'],
      dtype='object')

### Geocoding 

- In this example the Banks csv data  has multiple columns and we are using the :
    - Address, City, County, Stname and Zip columns for our geocoding
    - After those fields have been initialized, we use the UIGeocode to run the actual geocoding 
    - Here there are three arguments needed for the UIGeocode
      - The input CSV in this example Bank.csv
      - The address locator we are referencing.
      - And the name of the output feature class

In [5]:
G = UIG.Geocoder('ADDRESS','CITY','COUNTY','STNAME','ZIP')
G.UIGeocode('Bank.csv','CompositePointUI.loc','BanksUI')

Initializing the input address fields
Adding address fields to the address table
Done adding address fields 
Geocoding addressses with the CompositePointUI.loc address locator
The output feature class for this geocoding is BanksUI
251 were sucessfully geocoded
Start Time: Thursday, September 15, 2022 6:52:44 PM
Row Count = 251
Succeeded at Thursday, September 15, 2022 6:52:45 PM (Elapsed Time: 0.16 seconds)


In [6]:
Credit = pd.read_csv('CreditUnion.csv')
Credit.columns

Index(['CU_NUMBER', 'CYCLE_DATE', 'JOIN_NUMBER', 'SiteId', 'CU_NAME',
       'SiteName', 'SiteTypeName', 'MainOffice', 'PhysicalAddressLine1',
       'PhysicalAddressLine2', 'PhysicalAddressCity',
       'PhysicalAddressStateCode', 'PhysicalAddressPostalCode',
       'PhysicalAddressCountyName', 'PhysicalAddressCountry',
       'MailingAddressLine1', 'MailingAddressLine2', 'MailingAddressCity',
       'MailingAddressStateCode', 'MailingAddressStateName',
       'MailingAddressPostalCode', 'PhoneNumber', 'HoursOfOperation',
       'MemberServices', 'ATM', 'DriveThru'],
      dtype='object')

In [7]:
G = UIG.Geocoder('PhysicalAddressLine1','PhysicalAddressCity','PhysicalAddressCountyName',
                 'PhysicalAddressCountry','<None>')
G.UIGeocode('CreditUnion.csv','CompositePointUI.loc','CreditUI')

Initializing the input address fields
Adding address fields to the address table
Done adding address fields 
Geocoding addressses with the CompositePointUI.loc address locator
The output feature class for this geocoding is CreditUI
62 were sucessfully geocoded
Start Time: Thursday, September 15, 2022 6:52:54 PM
Row Count = 62
Succeeded at Thursday, September 15, 2022 6:52:55 PM (Elapsed Time: 0.16 seconds)


### Proximity

- In this example proximity to financial services is analyzed. Here the proximity class is initialized with 3 arguments:
 - The tax parcel data 
 - Proxmity variable 
 - Outputname for proximity analysis
 - The function is run like this : proximity('TaxData2021','BanksCreditUnionMerged','Banks_CreditUnion_2020')
 
 - if there are two or more feature classes that needs to be merged, like we have for this example for Banks and Credit Unions , the merged function is called.
 
 - The merge function is run like this merge('BanksUI','CreditUI')
 
 
 - A required new field is added to the new tax parcel data
   the function is run like this addfield('ResNearBanks')
 
 - Summary tables are created for all residential units and residential units that are near proximity variable using 
   the summary table. The method takes three arguments 
    - Residential units table 
    - All residenitial units tables
    - Geodatabase where table is exported 
   
 - The exportcsv method exports output of our proxmity analysis. It output location and name of the output csv as input
 
 
 


In [8]:
P = UIP.proximity('TaxData2021','BanksCreditUnionMerged','Banks_CreditUnion_2020')
P.merge('BanksUI','CreditUI')
P.addfield('ResNearBanks')
P.summarize('ParcelNPA2020','ALLNPA202',ProjectGDB)
P.exportcsv(path,'ProximityToFinancialServicesTest.csv')

Converting feature classes  into readble arcpy format
('BanksUI', 'CreditUI') will be merged and named BanksCreditUnionMerged
Merging Feature classes
Done merging
The Feature class has 313 features
Adding a fieldname ResNearBanks
Done adding field name
Running near analysis and filtering near parcels for summary
Calculating residential parcel within 0.5 miles of BanksCreditUnionMerged
Querying residential parcels the where near distance is greater than 0 
Total residential parcels selected: 76337
If near distance is greater than 0 ResNearBanks = 1


ExecuteError: ERROR 000574: Python is not installed.
ERROR 000574: Python is not installed.
ERROR 000574: Python is not installed.
ERROR 000574: Python is not installed.
ERROR 000574: Python is not installed.
ERROR 000574: Python is not installed.
ERROR 000574: Python is not installed.
Failed to execute (CalculateField).


In [ ]:
# Additonal Examples
# Proximity to Public Transit

P = UIP.proximity('TaxData2021','PublicTransport','PublicTransit')
P.merge('CATS_BusStops2019','GoldLine2016','LynxBlueLineStation')
P.addfield('ResNearTransit')
P.summarize('ParcelNPA2020','ALLNPA202',ProjectGDB)
P.exportcsv(path,'ProximityToTransit.csv')

In [ ]:
# Additonal Examples
# Proximity to Grocery


P = UIP.proximity('TaxData2021','GroceryStores2021','Grocery')
P.addfield('ResNearGrocery')
P.summarize('ParcelNPAUnits','ALLNPAUnits',ProjectGDB)
P.exportcsv(path,'ProximityToGrocery.csv')

### Impervious Surface

- The impervious surface method is initialized with three arguments :
    - Single family feature class 
    - commercial feature class 
    - Output name for union analyiss
- After initializing, the next step is to run the intersect method. The intersect method takes two arguments
    - NPA featureclass
    - Name of output feature class for intersect analysis
    
- The exportcsv is the last step for the intersect analysis. It takes two arguments. 
    - The output path of the intersect analysis
    - Name of the intersect input csv

In [ ]:
I = UIM.Impervious('Singlefamily_Impervious_2021','Commercial_Impervious_2021','Impervious2021')
I.intersect('NPA','IntersectImpervious2021')
I.exportcsv(path,'Impervious.csv')

### Aggregation

- The aggregate class is initialized with no arguments.This class has two methods:
    - The *withinNPA* method: This method aggregates point features classes that are completely contained by each NPA 
      The withinNPA method takes two arguments: 
       - NPA feature class
       - Point feature class to be aggregated
       - Name of output feature class
        
    - The *withNPAID* method: This method assign NPA IDs to point feature classes that are completely within each NPA
       - Point feature classes 
       - NPA feature class 
       - Name of output feature class

In [ ]:
A = UIA.aggregate()
A.withinNPA('NPA','BanksUI','BanksNPA')
A.withNPAID('CreditUI','NPA', 'CreditNPAID')